In [1]:
using Random
Random.seed!(1)

using Langevin
using Langevin.Geometries: Geometry
using Langevin.Lattices: Lattice
using Langevin.HolsteinModels: HolsteinModel
using Langevin.HolsteinModels: assign_μ!, assign_ω!, assign_λ!
using Langevin.HolsteinModels: assign_tij!, assign_ωij!
using Langevin.HolsteinModels: setup_checkerboard!, construct_expnΔτV!, mulMᵀ!,construct_M
using Langevin.InitializePhonons: init_phonons_single_site!
using Langevin.FourierAcceleration: FourierAccelerator
using Langevin.FourierAcceleration: forward_fft!, inverse_fft!, accelerate!, accelerate_noise!
using Langevin.LangevinDynamics: update_euler!, update_rk!, update_euler_fa!, update_rk_fa!


#############################
## DEFINING HOLSTEIN MODEL ##
#############################

# number of dimensions
ndim = 2 # 2 or 3 for square or cubic lattice

# number of orbitals per unit cell
norbits = 1

# lattice vectors
lvecs = [[1.0,0.0,0.0],
         [0.0,1.0,0.0]]
       #  [0.0,0.0,1.0]]

# basis vectors
bvecs = [[0.0,0.0,0.0]]

# defining square lattice geometry
geom = Geometry(ndim, norbits, lvecs, bvecs)

# defining lattice size
L = 6

# constructing finite square lattice
lattice = Lattice(geom,L)

# discretization
Δτ = 0.1

# setting temperature
β = 8.0

# constructing holstein model
holstein = HolsteinModel(geom,lattice,β,Δτ)

# defining nearest neighbor hopping on square lattice
t = 1.0
assign_tij!(holstein, t, 0.0, 1, 1, [1,0,0]) # x direction hopping
assign_tij!(holstein, t, 0.0, 1, 1, [0,1,0]) # y direction hopping
# assign_tij!(holstein, t, 0.0, 1, 1, [0,0,1]) # z direction hopping

# hamiltonian parameter values
ω = 1.0
λ = sqrt(2.0)
μ = -(λ/ω)^2 # for half-filling

assign_ω!(holstein, ω, 0.0)
assign_λ!(holstein, λ, 0.0)
assign_μ!(holstein, μ, 0.0)

# organize electron hoppings for checkerboard decomposition
setup_checkerboard!(holstein)

# intialize phonon field
holstein.ϕ .= -λ^2/ω^2
# r = rand(length(holstein))
# @. holstein.ϕ = (r-0.5)*(4.0/ω)

# construct exponentiated interaction matrix
construct_expnΔτV!(holstein)

####################################
## DEFINING SIMULATION PARAMETERS ##
####################################

# langevin time step
Δt = 1e-3

# tolerace of IterativeSolvers
tol = 1e-4

# mass for fourier acceleration: increasing the mass reduces the
# amount of acceleration
mass = 0.5

# number of thermalization steps
ntherm = 2000 # need 200_000 to reach SDW phase

# defining FourierAccelerator type
fa = FourierAccelerator(holstein,mass,Δt)


#################################################
## PRE-ALLOCATING ARRAYS NEEDED FOR SIMULATION ##
#################################################

dϕdt     = zeros(Float64,          length(holstein))
fft_dϕdt = zeros(Complex{Float64}, length(holstein))

dSdϕ     = zeros(Float64,          length(holstein))
fft_dSdϕ = zeros(Complex{Float64}, length(holstein))
dSdϕ2    = zeros(Float64,          length(holstein))

g    = zeros(Float64, length(holstein))
Mᵀg  = zeros(Float64, length(holstein))
M⁻¹g = zeros(Float64, length(holstein))

η     = zeros(Float64,          length(holstein))
fft_η = zeros(Complex{Float64}, length(holstein))

########################
## RUNNING SIMULATION ##
########################

# to store the number of IterativeSolver steps
iters = 0

# first do thermalization sweeps
for i in 1:ntherm
    
    # Runge-Kutta/Huen's Update with Fourier Acceleration.
    iters = update_rk_fa!(holstein, fa, dϕdt, fft_dϕdt, dSdϕ2, dSdϕ, fft_dSdϕ, g, Mᵀg, M⁻¹g, η, fft_η, Δt, tol)

end


┌ Info: Recompiling stale cache file /Users/kbarros/.julia/compiled/v1.1/Langevin/nUpf1.ji for Langevin [d56e79bc-9d16-11e9-1337-85b17f59915c]
└ @ Base loading.jl:1184


In [2]:
import Serialization
Serialization.serialize("holstein.dat", holstein)